In [1]:
!ls

batch_log.json	FindAnomoly.ipynb  path.png  stream_log.json


In [2]:
import pandas as pd

In [32]:
!head -n 10 stream_log.json

{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "5008", "amount": "47.99"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "7891", "amount": "65.22"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "1677", "amount": "70.52"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "4485", "amount": "40.75"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "5846", "amount": "18.55"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "8840", "amount": "89.90"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "6764", "amount": "29.74"}
{"event_type":"unfriend", "timestamp":"2017-06-14 18:46:50", "id1": "5256", "id2": "3150"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "2743", "amount": "117.64"}
{"event_type":"purchase", "timestamp":"2017-06-14 18:46:50", "id": "1111", "amount": "13.15"}


# Processing the Batch file

In [36]:
import json
import pandas as pd
import numpy as np

fileName='batch_log.json'
Param=[]
Purchase = []
NetworkBe=[]
NetworkNot=[]

with open(fileName) as f:
    for line in f:
            
        if line.startswith('{"D":'):
            LineElem=json.loads(line)
            D,T=int(LineElem['D']),int(LineElem['T'])
            Param=[D,T]
        elif line.startswith('{"event_type":"purchase"'):
            LineElem=json.loads(line)
            Purchase.append(np.array([LineElem["timestamp"],LineElem["id"],LineElem["amount"]]))
        elif line.startswith('{"event_type":"befriend"'):
            LineElem=json.loads(line)
            NetworkBe.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
        elif line.startswith('{"event_type":"unfriend"'):
            LineElem=json.loads(line)
            NetworkNot.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))


In [40]:
DF_Purch=pd.DataFrame(Purchase,columns=['timestamp','id','amount'])
DF_NetworkBe=pd.DataFrame(NetworkBe,columns=['timestamp','id1','id2'])
NetworkNot=pd.DataFrame(NetworkNot,columns=['timestamp','id1','id2'])


# Building scocial Network Graph

In [64]:
set1= set(DF_NetworkBe.id1.tolist())
set2= set(DF_NetworkBe.id2.tolist())
SetAll=set1.union(set2)
ListAll=list(SetAll)

DF_Id_list=DF_NetworkBe.groupby('id1')['id2'].apply(list).reset_index()

import networkx as nx
G=nx.Graph()
# add a list of nodes,
G.add_nodes_from(ListAll)

EdgesAdded=[(DF_NetworkBe['id1'][i],DF_NetworkBe['id2'][i]) for i in range(len(DF_NetworkBe))]

G.add_edges_from(EdgesAdded)

EdgesrEMOVED=[(NetworkNot['id1'][i],NetworkNot['id2'][i]) for i in range(len(NetworkNot))]

G.remove_edges_from(EdgesrEMOVED)

# Processing the real time stream file 

In [84]:
DD=list(nx.single_source_shortest_path_length(G, '649', cutoff=D).keys())

In [122]:
DF_Purch_i=DF_Purch.copy()

def isinNtetwork(x):
    if x in DD:
        return True
    else:
        return False

from datetime import datetime
import time

def Dt_to_UnixTime(dt):
    return time.mktime(dt.timetuple())
    

DF_Purch_i['InNetwork']= DF_Purch_i['id'].apply(isinNtetwork)
DF_Purch_i=DF_Purch_i[DF_Purch_i['InNetwork']==True].reset_index(drop=True)
DF_Purch_i=DF_Purch_i[['timestamp','id','amount']]
DF_Purch_i['timestamp'] = pd.to_datetime(DF_Purch_i['timestamp'])
DF_Purch_i['timestamp']=DF_Purch_i['timestamp'].apply(Dt_to_UnixTime)

In [143]:
DF_Purch_i.head()

timestamp    id  amount
0  1.497480e+09  8780  162.38
1  1.497480e+09  2131   58.98
2  1.497480e+09  3082  133.47
3  1.497480e+09  3517  145.49
4  1.497480e+09  7907   22.40

In [172]:
def getLastT(x):
    return x[-T:]

New1=DF_Purch_i.groupby('id')['timestamp'].apply(list).reset_index()
New2=DF_Purch_i.groupby('id')['amount'].apply(list).reset_index()

New3=New1.merge(New2)

In [174]:
T=3;
New3['timestamp']=New3['timestamp'].apply(getLastT)
New3['amount']=New3['amount'].apply(getLastT)

In [177]:
New3.head(10)

id                                   timestamp                   amount
0     1  [1497480408.0, 1497480409.0, 1497480409.0]    [25.83, 59.82, 67.74]
1  1000  [1497480409.0, 1497480409.0, 1497480409.0]   [37.23, 39.28, 167.67]
2  1028  [1497480409.0, 1497480409.0, 1497480410.0]  [173.91, 186.74, 28.37]
3  1086  [1497480409.0, 1497480409.0, 1497480409.0]   [106.89, 54.26, 75.72]
4  1109  [1497480409.0, 1497480409.0, 1497480409.0]    [75.52, 67.57, 17.51]
5   112  [1497480409.0, 1497480409.0, 1497480409.0]     [59.10, 8.86, 32.03]
6  1150  [1497480409.0, 1497480409.0, 1497480409.0]    [31.77, 25.62, 59.05]
7   117  [1497480408.0, 1497480408.0, 1497480409.0]   [33.73, 100.40, 69.22]
8  1208  [1497480409.0, 1497480409.0, 1497480409.0]     [40.02, 31.53, 9.70]
9  1230  [1497480409.0, 1497480409.0, 1497480409.0]   [73.53, 24.63, 121.82]

In [208]:
import numpy as np

def calcMean_std(DF):
    y=[]
    
    for i in range(len(DF)):
        y0=DF.amount[i]
        y0=[float(y0[j]) for j in range(len(y0))]
        y.extend(y0)
    return np.mean(y), np.std(y)


def isItAnom(DF,amount):
    mean,std=calcMean_std(DF)
    if (amount>mean+3*std):
        return True
    else:
        return False


In [209]:
calcMean_std(New3)

(65.85854561878952, 42.886136861715727)

In [210]:
isItAnom(New3,100)

False

In [211]:
isItAnom(New3,300)

True

In [171]:
LLL=New1.timestamp[New1['id']=='1'].tolist()[0]
getLastT(LLL)

[1497480407.0,
 1497480407.0,
 1497480407.0,
 1497480408.0,
 1497480408.0,
 1497480408.0,
 1497480408.0,
 1497480408.0,
 1497480409.0,
 1497480409.0]

In [145]:
New.head()


id                                          timestamp
0     1  [1497480400.0, 1497480400.0, 1497480401.0, 149...
1  1000  [1497480400.0, 1497480400.0, 1497480400.0, 149...
2  1028  [1497480400.0, 1497480400.0, 1497480400.0, 149...
3  1086  [1497480400.0, 1497480400.0, 1497480400.0, 149...
4  1109  [1497480400.0, 1497480401.0, 1497480401.0, 149...

In [127]:
DF_Purch_i.sort('timestamp', ascending=False)

/home/iabualhaol/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


timestamp    id  amount
14747  1.497480e+09  9188   19.56
14711  1.497480e+09  8089   17.72
14695  1.497480e+09  5467   29.05
14696  1.497480e+09  2624   53.97
14697  1.497480e+09  9522   43.98
14698  1.497480e+09  7482   19.73
14699  1.497480e+09  4194   32.19
14700  1.497480e+09  5867   70.54
14701  1.497480e+09  8108   97.20
14702  1.497480e+09  2467   64.60
14703  1.497480e+09  9434  159.56
14704  1.497480e+09  7786   64.17
14705  1.497480e+09  2890   18.02
14706  1.497480e+09  6356   23.21
14707  1.497480e+09  5388  146.20
14708  1.497480e+09  8089   25.59
14709  1.497480e+09  4194   70.39
14694  1.497480e+09  4029   68.07
14693  1.497480e+09  4560  119.28
14692  1.497480e+09  7687   36.31
14682  1.497480e+09  4472   16.68
14676  1.497480e+09  7694   53.04
14677  1.497480e+09  4257   22.22
14678  1.497480e+09  8993  108.60
14679  1.497480e+09  9053   36.13
14680  1.497480e+09  8276  136.60
14681  1.497480e+09   477   79.12
14683  1.497480e+09  2316   31.82
14691  1.497480e+09  9361   71.41
14684  1.497480e+09  8830   19.24
...             ...   ...     ...
855    1.497480e+09  6427  100.28
857    1.497480e+09  9361   12.19
829    1.497480e+09  4612  114.58
858    1.497480e+09  6873   76.73
859    1.497480e+09  4948   21.15
860    1.497480e+09  9780   75.52
861    1.497480e+09  8652   57.15
862    1.497480e+09  8052   62.83
863    1.497480e+09  9361   69.73
864    1.497480e+09  8789   46.76
848    1.497480e+09  9249   43.83
847    1.497480e+09  5017   74.07
846    1.497480e+09  5342   36.18
845    1.497480e+09  4754   28.27
830    1.497480e+09  7680   23.08
831    1.497480e+09  8295   26.88
832    1.497480e+09  8404   75.19
833    1.497480e+09  1512   39.76
834    1.497480e+09  3966   94.99
835    1.497480e+09  6873   43.32
836    1.497480e+09  1906   42.70
837    1.497480e+09  3972   50.45
838    1.497480e+09  1208   49.04
839    1.497480e+09  1439  125.73
840    1.497480e+09  7097   41.51
841    1.497480e+09  8680   31.90
842    1.497480e+09  8745    1.59
843    1.497480e+09  2695   64.09
844    1.497480e+09  1150   12.67
0      1.497480e+09  8780  162.38

[14748 rows x 3 columns]

In [128]:
DF_Purch_i[DF_Purch_i.id=='8780'].sort('timestamp', ascending=False)

/home/iabualhaol/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


timestamp    id  amount
13864  1.497480e+09  8780   63.99
12498  1.497480e+09  8780   16.29
12478  1.497480e+09  8780   72.62
12204  1.497480e+09  8780    1.01
11963  1.497480e+09  8780   84.71
11851  1.497480e+09  8780   41.16
11848  1.497480e+09  8780  116.55
11278  1.497480e+09  8780   77.06
10338  1.497480e+09  8780  119.98
11545  1.497480e+09  8780   22.72
10030  1.497480e+09  8780   80.11
9943   1.497480e+09  8780   41.85
9604   1.497480e+09  8780   12.43
9150   1.497480e+09  8780   32.73
9031   1.497480e+09  8780   78.89
7624   1.497480e+09  8780  121.46
8783   1.497480e+09  8780   52.36
8680   1.497480e+09  8780  120.79
8300   1.497480e+09  8780  127.01
8055   1.497480e+09  8780   16.33
7729   1.497480e+09  8780   41.95
6780   1.497480e+09  8780   77.52
6056   1.497480e+09  8780   62.26
5700   1.497480e+09  8780  116.81
5566   1.497480e+09  8780   11.46
5327   1.497480e+09  8780   23.96
5302   1.497480e+09  8780   25.87
5257   1.497480e+09  8780   59.84
4384   1.497480e+09  8780    2.15
3613   1.497480e+09  8780   30.87
2919   1.497480e+09  8780   25.74
2732   1.497480e+09  8780  105.65
2861   1.497480e+09  8780  136.13
4211   1.497480e+09  8780   21.76
3203   1.497480e+09  8780   97.06
4222   1.497480e+09  8780   64.14
2643   1.497480e+09  8780   69.84
2349   1.497480e+09  8780  154.82
2197   1.497480e+09  8780   35.98
1265   1.497480e+09  8780   65.28
1071   1.497480e+09  8780  165.43
406    1.497480e+09  8780   79.63
0      1.497480e+09  8780  162.38

In [104]:
from datetime import datetime
import time

dt = datetime.fromtimestamp(1346236702)
print (dt)
print (time.mktime(dt.timetuple()))




2012-08-29 06:38:22
1346236702.0


In [112]:
DF_Purch_i['timestamp'][0]

'2017-06-14 18:46:40'

In [121]:
Dt_to_UnixTime(DF_Purch_i['timestamp'][0])

1497480400.0

In [74]:
fileNameStream='stream_log.json'
Purchase2 = []
NetworkBe2=[]
NetworkNot2=[]

def addEdge(G,Edge):
    G.add_edges_from(Edge)

def rmEdge(G,Edge):
    G.remove_edges_from(Edge)
                     
def addNode(G,Node):
    G.add_node(Node)

def addNodes(G,Nodes):
    G.add_nodes_from(ListAll)
                    

# Param=[D,T]
# T and D are gllobal vraibles

def CheckAnomoly(timestamp,node_id,amount,T, D):
    # Build dataFrame with all id in the social network with D distance from node_id
    
    
    # Filter the graph with T nearest transactions to the time stamp 
    
    # calculate the mean and the Std of the network 
    
    # if the amount is > mean + 3*std  falg the transaction as Anomoly
    
    
    
        


with open(fileNameStream) as f:DF_Purch_i
    for line in f:
  
        if line.startswith('{"event_type":"purchase"'):
            LineElem=json.loads(line)
            Purchase2.append(np.array([LineElem["timestamp"],LineElem["id"],LineElem["amount"]]))
            addNode(G,LineElem["id"])
            CheckAnomoly(LineElem["timestamp"],LineElem["id"],LineElem["amount"])
           
        elif line.startswith('{"event_type":"befriend"'):
            LineElem=json.loads(line)
            NetworkBe2.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
            # Updating the scial network graph (adding)
            addNodes(G,[LineElem["id1"],LineElem["id2"]])
            addEdge(G,[(LineElem["id1"],LineElem["id2"])])

           
            
        elif line.startswith('{"event_type":"unfriend"'):
            LineEl4em=json.loads(line)
            NetworkNot2.append(np.array([LineElem["timestamp"],LineElem["id1"],LineElem["id2"]]))
            # Updating the scial network graph (removing) 
            addNodes(G,[LineElem["id1"],LineElem["id2"]])
            rmEdge(G,[(LineElem["id1"],LineElem["id2"])])
        
    
        

In [73]:
 addEdge(G,[(LineElem["id1"],LineElem["id2"])])

In [58]:
(NetworkBe2[i2][1],NetworkBe2[i2][2])


('3556', '2143')

In [5]:

DF_Purch_i

In [43]:
'''
G.number_of_nodes()
G.number_of_edges()

DF_Id_list.id2[DF_Id_list.id1=='1'].tolist()

DF_Id_list["Length"]=DF_Id_list.id2.apply(len)

len(DF_Id_list.id2[0])

G.neighbors('1')

nx.single_source_shortest_path_length(G, '1', cutoff=1)

nx.single_source_shortest_path_length(G, '1', cutoff=2)

%matplotlib inline
import matplotlib.pyplot as plt
nx.draw(G)
plt.show()
plt.savefig("path.png")
'''

'\nG.number_of_nodes()\nG.number_of_edges()\n\nDF_Id_list.id2[DF_Id_list.id1==\'1\'].tolist()\n\nDF_Id_list["Length"]=DF_Id_list.id2.apply(len)\n\nlen(DF_Id_list.id2[0])\n\nG.neighbors(\'1\')\n\n%matplotlib inline\nimport matplotlib.pyplot as plt\nnx.draw(G)\nplt.show()\nplt.savefig("path.png")\n'

In [31]:
nx.single_source_shortest_path_length(G, '1', cutoff=2)

{'1': 0,
 '1000': 2,
 '1038': 2,
 '104': 2,
 '1041': 2,
 '1048': 2,
 '1057': 2,
 '1070': 2,
 '1073': 2,
 '1075': 2,
 '1139': 2,
 '1169': 2,
 '1174': 2,
 '1201': 2,
 '1208': 2,
 '1355': 2,
 '1452': 2,
 '1491': 2,
 '1504': 2,
 '1578': 2,
 '1592': 2,
 '1603': 2,
 '1605': 2,
 '1689': 2,
 '1699': 2,
 '1710': 2,
 '1713': 2,
 '1844': 2,
 '187': 2,
 '1919': 2,
 '1948': 2,
 '1957': 1,
 '1958': 2,
 '1970': 2,
 '2020': 2,
 '2054': 2,
 '2109': 2,
 '2115': 2,
 '2139': 2,
 '2160': 2,
 '2178': 2,
 '2230': 2,
 '2245': 2,
 '2257': 2,
 '2267': 2,
 '23': 2,
 '2438': 2,
 '2542': 2,
 '2550': 1,
 '2570': 2,
 '269': 2,
 '2695': 2,
 '2720': 2,
 '2739': 2,
 '2745': 2,
 '2752': 2,
 '2765': 2,
 '2770': 1,
 '2809': 2,
 '2824': 2,
 '2853': 2,
 '2862': 2,
 '2875': 2,
 '2878': 2,
 '2894': 2,
 '2983': 2,
 '2997': 2,
 '303': 2,
 '3064': 2,
 '3102': 2,
 '3142': 2,
 '315': 2,
 '3153': 2,
 '3190': 2,
 '3267': 2,
 '3299': 2,
 '3340': 2,
 '3342': 2,
 '3390': 2,
 '3402': 2,
 '3477': 2,
 '3482': 2,
 '3484': 2,
 '355': 2,
 '3